In [1]:
from drl_agent import DRLAgent
from env import StockPortfolioEnv
from data_loader import Yahoo_Downloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
import pandas as pd
from utils import data_split
from pyfolio import timeseries

/Users/vsaravind/dev/rl_final_lab/portfolio_manager/.venv/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
from settings import MODEL_MAP

In [7]:
def get_data(tickers, start_date, end_date):
    df = Yahoo_Downloader(
        ticker_list=tickers, start_date=start_date, end_date=end_date
    ).fetch_data()
    fe = FeatureEngineer(
        use_technical_indicator=True, use_turbulence=False, user_defined_feature=False
    )
    df = fe.preprocess_data(df)

    # add covariance matrix as states
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in range(lookback, len(df.index.unique())):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)

        covs = return_lookback.cov().values
        cov_list.append(covs)

    df_cov = pd.DataFrame(
        {"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list}
    )
    df = df.merge(df_cov, on="date")
    df = df.sort_values(["date", "tic"]).reset_index(drop=True)
    df = data_split(df, start_date, end_date)
    return df

In [8]:
def generate_environment(
    df,
    initial_amount,
    transaction_cost_pct=0,
    tech_indicator_list=["macd", "rsi_30", "cci_30", "dx_30"],
):
    stock_dimension = len(df.tic.unique())
    state_space = stock_dimension
    env_kwargs = {
        "hmax": 100,
        "initial_amount": initial_amount,
        "transaction_cost_pct": transaction_cost_pct,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": tech_indicator_list,
        "action_space": stock_dimension,
        "reward_scaling": 1e-1,
    }
    e_gym = StockPortfolioEnv(df=df, **env_kwargs)
    return e_gym

In [9]:
def get_agent(env):
    return DRLAgent(env)

In [14]:
from settings import TICKERS, MODEL_PARAMS_MAP, MODEL_TRAINED_MAP

In [15]:
def get_model(model_name, agent):
    model_params = MODEL_PARAMS_MAP[model_name]
    model = agent.get_model(model_name, model_kwargs = model_params)
    model.load(MODEL_TRAINED_MAP[model_name])
    return model

In [11]:
dataset = get_data(tickers=TICKERS, start_date="2022-01-01", end_date="2023-12-31")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (15030, 8)
Successfully added technical indicators


In [20]:
env = generate_environment(dataset, initial_amount=1000000)

In [21]:
agent = get_agent(env)
model = get_model("ppo", agent)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.001, 'batch_size': 128}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
model

In [23]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model, environment=env)

begin_total_asset:1000000
end_total_asset:1153248.669765873
Sharpe:  1.3021361932379374
hit end!


In [ ]:
trained_ppo = agent.train_model(model=model, tb_log_name="ppo", total_timesteps=40000)

In [62]:
trained_ppo.save("ppo_dow30_2004_2021")

In [68]:
model = get_model("a2c", agent)

{'n_steps': 10, 'ent_coef': 0.005, 'learning_rate': 0.0004}
Using cpu device


In [ ]:
trained_a2c = agent.train_model(model=model, tb_log_name="a2c", total_timesteps=40000)

In [70]:
trained_a2c.save("a2c_dow30_2004_2021")

In [12]:
trained_ppo = model.load("ppo_dow30_2004_2021")

In [13]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_ppo, environment=env)

/Users/vsaravind/dev/rl_final_lab/portfolio_manager/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:7608291.88726857
Sharpe:  0.7586504043041269
hit end!


In [28]:
total_returns = 0
for i in range(len(df_daily_return_ppo)):
    total_returns += df_daily_return_ppo["daily_return"].iloc[i]

23229.57297948365

In [36]:
from utils import convert_daily_return_to_pyfolio_ts, extract_weights

In [58]:
def get_stats(predictions):
    stats = {}
    for prediction in predictions:
        daily_return = predictions[prediction]["daily_return"]
        pyfolio_ts = convert_daily_return_to_pyfolio_ts(daily_return)
        stats[prediction] = timeseries.perf_stats(
            returns=pyfolio_ts,
            factor_returns=pyfolio_ts,
            positions=None,
            transactions=None,
        )
    return stats

In [37]:
from pyfolio import timeseries

perf_func = timeseries.perf_stats
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)
perf_stats_all_ppo = perf_func(
    returns=DRL_strat_ppo,
    factor_returns=DRL_strat_ppo,
    positions=None,
    transactions=None,
    turnover_denom="AGB",
)

In [61]:
def get_profit(stats, env):
    profit = {}
    for prediction in stats:
        profit[prediction] = env.initial_amount * stats[prediction]["Cumulative returns"] / 100
    return profit

In [38]:
perf_stats_all_ppo

Annual return           0.135364
Cumulative returns      6.608292
Annual volatility       0.191563
Sharpe ratio            0.758650
Calmar ratio            0.298424
Stability               0.964357
Max drawdown           -0.453597
Omega ratio             1.165949
Sortino ratio           1.090486
Skew                    0.090470
Kurtosis               15.658521
Tail ratio              0.946131
Daily value at risk    -0.023558
Alpha                   0.000000
Beta                    1.000000
dtype: float64

In [45]:
1000000 * perf_stats_all_ppo["Cumulative returns"] / 100

66082.91887268586

In [51]:
import plotly.graph_objs as go

In [46]:
def get_prediction(models, env):
    result = {}
    for model in models:
        df_daily_return, df_actions = DRLAgent.DRL_prediction(model=model, environment=env)
        result[model.__class__.__name__] = {
            "daily_return": df_daily_return,
            "actions": df_actions,
        }
    return result

In [47]:
def prediction_plot(predictions):
    traces = []
    for prediction in predictions:
        df_daily_return = predictions[prediction]["daily_return"]
        df_cumprod = (df_daily_return.daily_return + 1).cumprod() - 1
        time_ind = pd.Series(df_daily_return.date)
        trace_portfolio = go.Scatter(x=time_ind, y=df_cumprod, mode="lines", name=prediction)
        traces.append(trace_portfolio)

    fig = go.Figure()
    for trace in traces:
        fig.add_trace(trace)
    fig.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(family="sans-serif", size=15, color="black"),
            bgcolor="White",
            bordercolor="white",
            borderwidth=2,
        ),
    )
    fig.update_layout(
        title={
            'text': "Cumulative Return",
            "y": 0.85,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        }
    )

    fig.update_layout(
        paper_bgcolor="rgba(1,1,0,0)",
        plot_bgcolor="rgba(1, 1, 0, 0)",
        xaxis_title="Date",
        yaxis=dict(titlefont=dict(size=30), title="Cumulative Return"),
        font=dict(
            size=40,
        ),
    )
    fig.update_layout(font_size=20)
    fig.update_traces(line=dict(width=2))

    fig.update_xaxes(
        showline=True,
        linecolor="black",
        showgrid=True,
        gridwidth=1,
        gridcolor="LightSteelBlue",
        mirror=True,
    )
    fig.update_yaxes(
        showline=True,
        linecolor="black",
        showgrid=True,
        gridwidth=1,
        gridcolor="LightSteelBlue",
        mirror=True,
    )
    fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor="LightSteelBlue")
    return fig

In [49]:
predictions = get_prediction([trained_ppo], env)

/Users/vsaravind/dev/rl_final_lab/portfolio_manager/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:7608291.88726857
Sharpe:  0.7586504043041269
hit end!


In [73]:
def get_splits(predictions, amount):
    splits = {}
    for prediction in predictions:
        action_df = predictions[prediction]["actions"]
        splits[prediction] = action_df.multiply(amount)
    return splits

In [26]:
stats = get_stats(predictions)

NameError: name 'get_stats' is not defined

In [66]:
get_profit(stats, env)

{'PPO': 66082.91887268586}

In [74]:
splits = get_splits(predictions, 1000000)

In [25]:
stats

NameError: name 'stats' is not defined

In [77]:
splits["PPO"]

,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2005-01-03,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,...,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037,37037.037037
2005-01-04,36966.472864,36985.516548,37115.294486,36966.472864,36978.222430,36966.472864,37293.922156,36966.472864,36966.472864,37019.569427,...,36966.472864,36966.472864,36966.472864,37201.266736,36966.472864,36966.472864,36966.472864,36966.472864,37069.808692,37248.853594
2005-01-05,37050.403655,36944.624037,36944.624037,36979.626864,36944.624037,36944.624037,37300.128490,36944.624037,36944.624037,36944.624037,...,36944.624037,36944.624037,36983.117461,36944.624037,36944.624037,37153.344601,36944.624037,37053.670734,36944.624037,37165.004760
2005-01-06,37129.126489,36947.712302,36947.712302,37313.770503,36947.712302,36947.712302,37315.003574,37066.515535,36947.712302,36947.712302,...,36947.712302,36947.712302,37007.987499,37048.012018,36947.712302,37011.004984,36947.712302,37153.620273,37011.854351,37161.361426
2005-01-07,36982.148886,36953.322589,36953.322589,37274.457514,37115.067244,36953.322589,37106.376141,37074.372172,36953.322589,36953.322589,...,37107.501179,36953.322589,37012.841552,36953.322589,36953.322589,37039.715797,36953.322589,37166.170776,36953.322589,36953.322589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,36954.514682,37027.396262,36954.514682,37027.560174,36954.514682,36954.514682,36954.514682,37009.827793,37265.487015,37194.401026,...,36954.514682,36954.514682,37002.146244,37332.434207,37265.747786,36954.514682,36954.514682,36954.514682,37233.728915,37164.349109
2020-12-28,36938.600242,36999.341100,36938.600242,36960.206926,36938.600242,37040.587515,36938.600242,36998.376250,37143.815309,37113.677710,...,36938.600242,36938.600242,37080.623209,37339.612842,37244.532257,36938.600242,36938.600242,36938.600242,37200.339139,37048.015743
2020-12-29,36934.819072,36940.746009,36934.819072,36943.115294,36934.819072,36934.819072,36934.819072,36934.819072,37101.715803,36934.819072,...,37067.450583,36934.819072,36934.819072,37231.158465,37089.627236,36934.819072,37161.279470,36934.819072,37185.251713,37298.686802


In [36]:
pd.DataFrame(dataset['open']).T

,0,0,0,0,0,0,0,0,0,0,...,248,248,248,248,248,248,248,248,248,248
open,126.889999,263.01001,148.559998,195.179993,240.0,141.220001,48.279999,170.949997,90.0,51.66,...,108.989998,376.0,108.959999,146.0,189.339996,525.97998,260.570007,37.380001,26.440001,52.509998


In [52]:
prediction_plot(predictions)